In [58]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pandas_profiling as ppf
# from tensorboardX import SummaryWriter
from sklearn.model_selection import train_test_split
# writer=SummaryWriter(logdir='./logistic')
torch.__version__

## logistic回归介绍


logistic回归是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。它们的模型形式基本上相同，都具有 wx + b，其中w和b是待求参数，其区别在于他们的因变量不同，多重线性回归直接将wx+b作为因变量，即y =wx+b,而logistic回归则通过函数L将wx+b对应一个隐状态p，p =L(wx+b),然后根据p 与1-p的大小决定因变量的值。如果L是logistic函数，就是logistic回归，如果L是多项式函数就是多项式回归。

说的更通俗一点，就是logistic回归会在线性回归后再加一层logistic函数的调用。

logistic回归主要是进行二分类预测，我们在激活函数时候讲到过 Sigmod函数，Sigmod函数是最常见的logistic函数，因为Sigmod函数的输出的是是对于0~1之间的概率值，当概率大于0.5预测为1，小于0.5预测为0。

以下为僵尸企业分类的逻辑回归实践

In [46]:
encoded_all_data=pd.read_csv(r'./data/created_data/encoded_all_data.csv').reset_index(drop=True).set_index('ID')
init_x=encoded_all_data.dropna(subset=['flag']).drop(['flag'],axis=1)
init_y=encoded_all_data.dropna(subset=['flag'])['flag']
# encoded_all_data

In [47]:
train_X,val_X, train_y, val_y = train_test_split(init_x,init_y,test_size=0.30, random_state=0)
display(train_X,train_y)
display(val_X,val_y)
val_X.dtypes

,专利,企业类型,区域,商标,控制人持股比例,控制人类型,注册时间,注册资本,著作权,行业,...,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
ID,,,,,,,,,,,,,,,,,,,,,
5989257.0,0.0,1,3,0.0,0.70,1,4.0,-0.973541,0.0,4,...,0.135151,0.563362,-0.536135,-0.609535,-0.405129,-0.448206,-0.248830,-0.204542,0.124025,0.432645
5991406.0,0.0,3,6,1.0,0.79,1,11.0,-0.254808,0.0,1,...,-0.340444,-0.212910,-0.256340,-0.397152,-0.520638,-0.499634,-0.355941,0.082914,-0.157695,0.399039
5987623.0,1.0,1,4,1.0,0.90,0,10.0,1.179153,0.0,3,...,-0.116104,0.063135,-0.270980,-0.065330,-0.117421,-0.086136,-0.252447,0.356294,-0.020507,-0.221159
185734.0,1.0,0,5,0.0,0.85,1,5.0,-0.447639,0.0,3,...,0.575990,-0.932660,0.018246,0.396604,0.095771,-0.094450,0.064934,-0.574421,-0.463028,-0.694906
5978163.0,0.0,4,1,1.0,0.85,0,4.0,-0.128591,1.0,3,...,-0.340444,-0.088663,-0.034559,-0.045658,-0.416634,-0.435343,-0.210012,-0.090157,-0.106243,0.044631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5985562.0,1.0,1,1,0.0,0.81,1,6.0,0.611179,1.0,1,...,-0.340444,-0.556945,0.183051,-0.071368,-0.000516,-0.009069,0.379197,-0.270317,-0.232909,0.345724
5997416.0,0.0,4,6,0.0,0.52,1,9.0,-0.054965,0.0,1,...,0.179926,-0.310291,-0.098579,-0.079990,0.037919,0.089179,-0.357966,-0.171569,-0.044199,0.027948
5995645.0,1.0,2,3,0.0,0.60,0,8.0,-0.184687,1.0,5,...,0.594195,0.544538,-0.037865,0.313914,-0.341026,-0.279906,-0.262372,0.078621,-0.152383,-0.611328


ID
5989257.0    0.0
5991406.0    0.0
5987623.0    0.0
185734.0     1.0
5978163.0    0.0
            ... 
5985562.0    0.0
5997416.0    0.0
5995645.0    0.0
5986066.0    0.0
600900.0     1.0
Name: flag, Length: 24208, dtype: float64

,专利,企业类型,区域,商标,控制人持股比例,控制人类型,注册时间,注册资本,著作权,行业,...,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
ID,,,,,,,,,,,,,,,,,,,,,
5995059.0,0.0,4,5,0.0,0.92,1,12.0,0.148384,0.0,4,...,-0.230072,-1.561244,-0.510114,-0.364237,-0.328120,-0.256488,-0.335577,-0.043331,0.063264,0.022658
2218002.0,0.0,1,3,0.0,0.93,0,4.0,-1.646697,0.0,4,...,-0.340444,0.506850,-0.953860,-0.779825,-0.740366,-0.697682,-0.614383,-0.224549,-0.463028,0.220451
5995763.0,0.0,0,6,1.0,0.96,0,12.0,1.084491,1.0,1,...,0.078421,0.199565,-0.378664,-0.309534,-0.195477,-0.250958,-0.254380,0.005627,-0.026336,0.091984
5980407.0,0.0,2,3,1.0,0.70,0,9.0,-0.475687,1.0,2,...,0.230234,0.744363,-0.190824,-0.460962,-0.456403,-0.444980,-0.325580,-0.058428,-0.294665,0.594452
5993287.0,0.0,3,6,0.0,0.56,0,4.0,-1.310119,0.0,4,...,-0.136206,-0.835341,-0.547950,-0.423297,-0.601732,-0.548048,-0.551252,0.024384,-0.339982,0.078504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993392.0,0.0,2,6,0.0,0.58,0,13.0,-0.738638,0.0,3,...,-0.285129,-0.435663,-0.814606,-0.678892,-0.687574,-0.674051,-0.604358,-0.206374,-0.438804,0.210009
405953.0,0.0,4,2,0.0,0.98,0,4.0,-0.338952,0.0,0,...,-0.340444,0.385515,0.557201,0.427475,0.022675,0.158300,0.191022,-0.540022,-0.463028,-0.077693
3384114.0,0.0,1,4,1.0,0.81,0,7.0,0.516516,1.0,4,...,1.926933,0.210563,0.589520,0.280960,0.865804,0.651634,1.113763,-0.896597,-0.463028,0.224107


ID
5995059.0    0.0
2218002.0    1.0
5995763.0    0.0
5980407.0    0.0
5993287.0    0.0
            ... 
5993392.0    0.0
405953.0     1.0
3384114.0    1.0
5980739.0    0.0
5993812.0    0.0
Name: flag, Length: 10376, dtype: float64

专利             float64
企业类型             int64
区域               int64
商标             float64
控制人持股比例        float64
控制人类型            int64
注册时间           float64
注册资本           float64
著作权            float64
行业               int64
债权融资额度         float64
债权融资成本         float64
股权融资额度         float64
股权融资成本         float64
内部融资和贸易融资额度    float64
内部融资和贸易融资成本    float64
项目融资和政策融资额度    float64
项目融资和政策融资成本    float64
从业人数           float64
资产总额           float64
负债总额           float64
营业总收入          float64
主营业务收入         float64
利润总额           float64
净利润            float64
纳税总额           float64
所有者权益合计        float64
dtype: object

#### logistic regression model

In [48]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=nn.Linear(27,2) # 由于16个维度已经固定了，所以这里写16
    def forward(self,x):
        x=self.fc(x)
        out=torch.sigmoid(x)
        return out


def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [49]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [50]:
device = get_default_device()
device

device(type='cuda')

In [51]:
# 网络模型实例化
net=LR()
to_device(net, device)
# 使用CrossEntropyLoss损失
criterion=nn.CrossEntropyLoss() # BCEloss待测
# 优化器
optm=torch.optim.Adamax(net.parameters(),lr=0.01) 
# 训练1500次
epochs=10000

#### train model

In [52]:
for i in range(epochs):
#      指定模型为训练模式，计算梯度
    net.train()
#      输入值都需要转化成torch的Tensor
    x=to_device(torch.from_numpy(train_X.values).float(),device)
    y=to_device(torch.from_numpy(train_y.values).long(),device)
#     输入数据
    y_hat=net(x)
#     计算损失
    loss=criterion(y_hat,y) 
#     前一步的损失清零
    optm.zero_grad() 
#     反向传播 
    loss.backward() 
#     更新参数
    optm.step() 
#     writer.add_scalar('tra_loss',loss,i)

#     if True:
    if (i+1)%200 ==0 : # 这里我们每100次输出相关的信息
#         指定模型为计算模式
        net.eval()
        test_in=to_device(torch.from_numpy(val_X.values).float(),device)
        test_l=to_device(torch.from_numpy(val_y.values).long(),device)
        test_out=net(test_in)
#          使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
#         writer.add_scalar('tes_loss',loss.item(),i)
#         writer.add_scalar('tes_acc',accu,i)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))    

Epoch:200,Loss:0.5351,Accuracy：0.84
Epoch:400,Loss:0.4955,Accuracy：0.89
Epoch:600,Loss:0.4732,Accuracy：0.92
Epoch:800,Loss:0.4576,Accuracy：0.93
Epoch:1000,Loss:0.4454,Accuracy：0.94
Epoch:1200,Loss:0.4354,Accuracy：0.94
Epoch:1400,Loss:0.4269,Accuracy：0.95
Epoch:1600,Loss:0.4194,Accuracy：0.95
Epoch:1800,Loss:0.4128,Accuracy：0.95
Epoch:2000,Loss:0.4068,Accuracy：0.95
Epoch:2200,Loss:0.4014,Accuracy：0.96
Epoch:2400,Loss:0.3964,Accuracy：0.96
Epoch:2600,Loss:0.3918,Accuracy：0.96
Epoch:2800,Loss:0.3876,Accuracy：0.96
Epoch:3000,Loss:0.3837,Accuracy：0.96
Epoch:3200,Loss:0.3800,Accuracy：0.96
Epoch:3400,Loss:0.3767,Accuracy：0.96
Epoch:3600,Loss:0.3736,Accuracy：0.96
Epoch:3800,Loss:0.3706,Accuracy：0.96
Epoch:4000,Loss:0.3680,Accuracy：0.96
Epoch:4200,Loss:0.3655,Accuracy：0.96
Epoch:4400,Loss:0.3632,Accuracy：0.96
Epoch:4600,Loss:0.3611,Accuracy：0.96
Epoch:4800,Loss:0.3592,Accuracy：0.96
Epoch:5000,Loss:0.3574,Accuracy：0.96
Epoch:5200,Loss:0.3558,Accuracy：0.97
Epoch:5400,Loss:0.3543,Accuracy：0.97
Epoch

In [53]:
# writer.close()

#### 为无标签的数据标上flag

In [54]:
non_label=encoded_all_data.drop(init_x.index)
non_label_X=non_label.drop(['flag'],axis=1)
estimate_y=estimate_y=np.array([1]*non_label_X.shape[0])
non_label_X

,专利,企业类型,区域,商标,控制人持股比例,控制人类型,注册时间,注册资本,著作权,行业,...,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
ID,,,,,,,,,,,,,,,,,,,,,
1080523.0,0.0,0,1,0.0,0.57,0,0.0,-0.254808,0.0,4,...,-0.340444,-0.351677,0.184651,0.417659,-0.258273,-0.272971,-0.235634,-0.424045,-0.463028,-0.522521
1080756.0,0.0,0,0,0.0,0.79,0,5.0,1.158117,0.0,5,...,-0.340444,-0.376625,1.042700,1.238097,0.496213,0.527454,1.150685,-0.738470,-0.463028,-0.921111
1080951.0,1.0,2,4,1.0,0.93,0,4.0,0.463926,0.0,2,...,0.510628,0.342302,0.870810,0.557917,0.176418,0.061825,0.351885,-0.604207,-0.463028,0.077015
1080972.0,0.0,4,4,0.0,0.58,1,0.0,-1.699287,0.0,1,...,-0.340444,0.149991,-0.955436,-0.773011,-0.726476,-0.690639,-0.618036,-0.230269,-0.463028,0.205945
1081027.0,1.0,1,6,1.0,0.72,1,12.0,1.231744,0.0,5,...,-0.035733,0.421098,-0.210764,-0.388153,-0.282905,-0.202919,-0.323033,-0.415748,-0.463028,0.438726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999614.0,1.0,4,0,1.0,0.51,0,2.0,-0.563337,0.0,2,...,0.143578,-0.266909,0.724965,-0.034803,0.071062,-0.043789,0.206325,0.250782,0.157708,0.982942
5999746.0,0.0,0,3,0.0,0.91,1,11.0,0.842576,0.0,2,...,-0.340444,-0.485126,-0.465091,-0.262250,-0.274809,-0.346624,-0.239135,0.038662,0.204388,-0.104008
5999945.0,0.0,3,4,1.0,0.83,0,13.0,-1.646697,1.0,4,...,-0.340444,-0.433419,-0.970887,-0.810904,-0.761767,-0.724710,-0.644649,-0.191496,-0.449280,0.255177


In [55]:
# net.eval()
# test_in=to_device(torch.from_numpy(non_label_X.values).float(),device)
# test_out=net(test_in)
# new_label=test_out.max(-1)[1].tolist()
new_label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [56]:
non_label_X['flag']=new_label
non_label_X

,专利,企业类型,区域,商标,控制人持股比例,控制人类型,注册时间,注册资本,著作权,行业,...,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计,flag
ID,,,,,,,,,,,,,,,,,,,,,
1080523.0,0.0,0,1,0.0,0.57,0,0.0,-0.254808,0.0,4,...,-0.351677,0.184651,0.417659,-0.258273,-0.272971,-0.235634,-0.424045,-0.463028,-0.522521,1
1080756.0,0.0,0,0,0.0,0.79,0,5.0,1.158117,0.0,5,...,-0.376625,1.042700,1.238097,0.496213,0.527454,1.150685,-0.738470,-0.463028,-0.921111,1
1080951.0,1.0,2,4,1.0,0.93,0,4.0,0.463926,0.0,2,...,0.342302,0.870810,0.557917,0.176418,0.061825,0.351885,-0.604207,-0.463028,0.077015,1
1080972.0,0.0,4,4,0.0,0.58,1,0.0,-1.699287,0.0,1,...,0.149991,-0.955436,-0.773011,-0.726476,-0.690639,-0.618036,-0.230269,-0.463028,0.205945,1
1081027.0,1.0,1,6,1.0,0.72,1,12.0,1.231744,0.0,5,...,0.421098,-0.210764,-0.388153,-0.282905,-0.202919,-0.323033,-0.415748,-0.463028,0.438726,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999614.0,1.0,4,0,1.0,0.51,0,2.0,-0.563337,0.0,2,...,-0.266909,0.724965,-0.034803,0.071062,-0.043789,0.206325,0.250782,0.157708,0.982942,0
5999746.0,0.0,0,3,0.0,0.91,1,11.0,0.842576,0.0,2,...,-0.485126,-0.465091,-0.262250,-0.274809,-0.346624,-0.239135,0.038662,0.204388,-0.104008,0
5999945.0,0.0,3,4,1.0,0.83,0,13.0,-1.646697,1.0,4,...,-0.433419,-0.970887,-0.810904,-0.761767,-0.724710,-0.644649,-0.191496,-0.449280,0.255177,0


In [60]:
logistic_final=encoded_all_data.combine_first(non_label_X)
logistic_final.to_csv(r'./data/created_data/logistic_final.csv')

In [59]:
# ppf.ProfileReport(logistic_final)